Analysis for Beneficial Microbes presentation

Still preliminary and needs tweaks

## Reimport scrubbed qiime table

In [ ]:
conda activate qiime2-2022.8

project=/Users/mallott/Dropbox/Projects/REACH/BM_presentation_analysis

qiime tools import \
    --input-path ${project}/fecal/decontaminated_samples_22_23.biom \
    --type 'FeatureTable[Frequency]' \
    --input-format BIOMV100Format \
    --output-path ${project}/fecal/scrubbed_table.qza

qiime feature-table summarize \
    --i-table ${project}/fecal/scrubbed_table.qza \
    --m-sample-metadata-file ${project}/fecal/metadata-combined.txt \
    --o-visualization ${project}/fecal/scrubbed_table.qzv

## Determine cutoffs for each sample type

I split the fecal samples into their own table and the soil samples into their own table. Then I created visualizations for both classified tables, built phylogenetic trees, and ran alpha rarefaction on both to determine rarefaction cutoffs.

In [ ]:
qiime feature-table filter-samples \
    --i-table ${project}/fecal/scrubbed_table.qza \
    --m-metadata-file ${project}/fecal/metadata-combined.txt \
    --p-where "[Type]='Feces'" \
    --o-filtered-table ${project}/fecal/scrubbed_fecal_table.qza

qiime feature-table summarize \
    --i-table ${project}/fecal/scrubbed_fecal_table.qza \
    --m-sample-metadata-file ${project}/fecal/metadata-combined.txt \
    --o-visualization ${project}/fecal/scrubbed_fecal_table.qzv

qiime phylogeny align-to-tree-mafft-fasttree \
    --i-sequences ${project}/fecal/rep-seqs-combined-1.qza \
    --o-alignment ${project}/fecal/aligned-rep-seqs-fecal.qza \
    --o-masked-alignment ${project}/fecal/masked-aligned-rep-seqs-fecal.qza \
    --o-tree ${project}/fecal/unrooted-tree-fecal.qza \
    --o-rooted-tree ${project}/fecal/rooted-tree-fecal.qza \
    --p-n-threads 3

qiime diversity alpha-rarefaction \
    --i-table ${project}/fecal/scrubbed_fecal_table.qza \
    --i-phylogeny ${project}/fecal/rooted-tree-fecal.qza \
    --p-max-depth 10000 \
    --m-metadata-file ${project}/fecal/metadata-combined.txt \
    --o-visualization ${project}/fecal/alpha-rarefaction-fecal.qzv

qiime feature-table filter-samples \
    --i-table ${project}/fecal/scrubbed_table.qza \
    --m-metadata-file ${project}/fecal/metadata-combined.txt \
    --p-where "[Type]='Soil'" \
    --o-filtered-table ${project}/soil/scrubbed_soil_table.qza

qiime feature-table filter-samples \
    --i-table ${project}/fecal/scrubbed_soil_table.qza \
    --m-metadata-file ${project}/soil/Combined_16S_Metadata.txt \
    --p-where "[Site]='Illinois'" \
    --o-filtered-table ${project}/soil/filtered_soil_table_il.qza

qiime feature-table filter-samples \
    --i-table ${project}/soil/filtered_soil_table_il.qza \
    --m-metadata-file ${project}/soil/Combined_16S_Metadata.txt \
    --p-where "[Year]='2023'" \
    --o-filtered-table ${project}/soil/filtered_soil_table_il_2023.qza

qiime feature-table summarize \
    --i-table ${project}/soil/filtered_soil_table_il_2023.qza \
    --m-sample-metadata-file ${project}/soil/Combined_16S_Metadata.txt \
    --o-visualization ${project}/soil/filtered_soil_table_il_2023.qzv

qiime phylogeny align-to-tree-mafft-fasttree \
    --i-sequences ${project}/fecal/rep-seqs-combined-1.qza \
    --o-alignment ${project}/soil/aligned-rep-seqs-soil.qza \
    --o-masked-alignment ${project}/soil/masked-aligned-rep-seqs-soil.qza \
    --o-tree ${project}/soil/unrooted-tree-soil.qza \
    --o-rooted-tree ${project}/soil/rooted-tree-soil.qza \
    --p-n-threads 6

qiime diversity alpha-rarefaction \
    --i-table ${project}/soil/filtered_soil_table_il_2023.qza \
    --i-phylogeny ${project}/soil/rooted-tree-soil.qza \
    --p-max-depth 20000 \
    --m-metadata-file ${project}/soil/Combined_16S_Metadata.txt \
    --o-visualization ${project}/soil/alpha-rarefaction-soil-il2023.qzv

qiime diversity alpha-rarefaction \
    --i-table ${project}/soil/filtered_soil_table_il_2023.qza \
    --i-phylogeny ${project}/soil/rooted-tree-soil.qza \
    --p-max-depth 7500 \
    --m-metadata-file ${project}/soil/Combined_16S_Metadata.txt \
    --o-visualization ${project}/soil/alpha-rarefaction-soil-il2023-7500.qzv

#### Fecal sample cutoffs
Shannon diversity levels off pretty quickly (>1000), but Faith's PD and observed features don't start leveling off until >4000. Rarefying to 1075 loses 1 sample from Illinois and 8 from Mississippi (total n after rarefying = 220). Rarefying to 4015 loses 15 from Illinois and 38 from Mississippi (total n after rarefying = 176). Probably need to run things both ways to be safe.

#### Soil sample cutoffs
3900 probably

### Diversity Indices

In [ ]:
qiime diversity core-metrics-phylogenetic \
  --i-phylogeny ${project}/fecal/rooted-tree-fecal.qza \
  --i-table ${project}/fecal/scrubbed_fecal_table.qza \
  --p-sampling-depth 4015 \
  --m-metadata-file ${project}/fecal/metadata-combined.txt \
  --output-dir ${project}/fecal/core-metrics-results-4015

qiime diversity core-metrics-phylogenetic \
  --i-phylogeny ${project}/fecal/rooted-tree-fecal.qza \
  --i-table ${project}/fecal/scrubbed_fecal_table.qza \
  --p-sampling-depth 1075 \
  --m-metadata-file ${project}/fecal/metadata-combined.txt \
  --output-dir ${project}/fecal/core-metrics-results-1075

qiime diversity core-metrics-phylogenetic \
  --i-phylogeny ${project}/soil/rooted-tree-soil.qza \
  --i-table ${project}/soil/filtered_soil_table_il_2023.qza \
  --p-sampling-depth 3900 \
  --m-metadata-file ${project}/soil/Combined_16S_Metadata.txt \
  --output-dir ${project}/soil/core-metrics-results-3900

qiime diversity core-metrics-phylogenetic \
  --i-phylogeny ${project}/soil/rooted-tree-soil.qza \
  --i-table ${project}/soil/filtered_soil_table_il_2023.qza \
  --p-sampling-depth 6050 \
  --m-metadata-file ${project}/soil/Combined_16S_Metadata.txt \
  --output-dir ${project}/soil/core-metrics-results-6050

### Taxonomic classification
Then I used the Greengenes2 Naive Bayesian Classifer.

In [ ]:
qiime feature-classifier classify-sklearn \
    --i-classifier 2022.10.backbone.full-length.nb.qza \
    --i-reads ${project}/fecal/rep-seqs-combined-1.qza \
    --o-classification ${project}/fecal/taxonomy-fecal.qza \
    --p-n-jobs 4

qiime metadata tabulate \
  --m-input-file ${project}/fecal/taxonomy-fecal.qza \
  --o-visualization ${project}/fecal/taxonomy-fecal.qzv

qiime taxa barplot \
  --i-table ${project}/fecal/scrubbed_fecal_table.qza \
  --i-taxonomy ${project}/fecal/taxonomy-fecal.qza \
  --m-metadata-file ${project}/fecal/metadata-combined.txt \
  --o-visualization ${project}/fecal/taxa-bar-plots-fecal.qzv

qiime taxa barplot \
  --i-table ${project}/soil/filtered_soil_table_il_2023.qza \
  --i-taxonomy ${project}/fecal/taxonomy-fecal.qza \
  --m-metadata-file ${project}/soil/Combined_16S_Metadata.txt \
  --o-visualization ${project}/soil/taxa-bar-plots-soil.qzv

### Extract various things for analysis
I then extracted alpha diversity indices, beta diversity indices, and feature tables for downstream analysis in R.

In [ ]:
qiime tools export --input-path ${project}/fecal/core-metrics-results-4015/shannon_vector.qza \
    --output-path ${project}/fecal/shannon
qiime tools export --input-path ${project}/fecal/core-metrics-results-4015/evenness_vector.qza \
    --output-path ${project}/fecal/evenness
qiime tools export --input-path ${project}/fecal/core-metrics-results-4015/faith_pd_vector.qza \
    --output-path ${project}/fecal/faith_pd
qiime tools export --input-path ${project}/fecal/core-metrics-results-4015/observed_features_vector.qza \
    --output-path ${project}/fecal/observed
cp ${project}/fecal/shannon/alpha-diversity.tsv ${project}/fecal/shannon-alpha-diversity.tsv
cp ${project}/fecal/evenness/alpha-diversity.tsv ${project}/fecal/evenness-alpha-diversity.tsv
cp ${project}/fecal/faith_pd/alpha-diversity.tsv ${project}/fecal/faithpd-alpha-diversity.tsv
cp ${project}/fecal/observed/alpha-diversity.tsv ${project}/fecal/observed-alpha-diversity.tsv

qiime tools export \
    --input-path ${project}/fecal/core-metrics-results-4015/unweighted_unifrac_distance_matrix.qza \
    --output-path ${project}/fecal/core-metrics-results-4015/extracted_unweighted_unifrac
cp ${project}/fecal/core-metrics-results-4015/extracted_unweighted_unifrac/distance-matrix.tsv \
    ${project}/fecal/unweighted-distance-matrix-4015.tsv
qiime tools export \
    --input-path ${project}/fecal/core-metrics-results-4015/weighted_unifrac_distance_matrix.qza \
    --output-path ${project}/fecal/core-metrics-results-4015/extracted_weighted_unifrac
cp ${project}/fecal/core-metrics-results-4015/extracted_weighted_unifrac/distance-matrix.tsv \
    ${project}/fecal/weighted-distance-matrix-4015.tsv
qiime tools export \
    --input-path ${project}/fecal/core-metrics-results-4015/bray_curtis_distance_matrix.qza \
    --output-path ${project}/fecal/core-metrics-results-4015/extracted_bray_curtis
cp ${project}/fecal/core-metrics-results-4015/extracted_bray_curtis/distance-matrix.tsv \
    ${project}/fecal/bray-distance-matrix-4015.tsv
qiime tools export \
    --input-path ${project}/fecal/core-metrics-results-4015/jaccard_distance_matrix.qza \
    --output-path ${project}/fecal/core-metrics-results-4015/extracted_jaccard
cp ${project}/fecal/core-metrics-results-4015/extracted_jaccard/distance-matrix.tsv \
    ${project}/fecal/jaccard-distance-matrix-4015.tsv

qiime taxa collapse \
    --i-table ${project}/fecal/scrubbed_fecal_table.qza \
    --i-taxonomy ${project}/fecal/taxonomy-fecal.qza \
    --p-level 2 --o-collapsed-table ${project}/fecal/scrubbed_fecal_table_phyla.qza
qiime taxa collapse \
    --i-table ${project}/fecal/scrubbed_fecal_table.qza \
    --i-taxonomy ${project}/fecal/taxonomy-fecal.qza \
    --p-level 5 --o-collapsed-table ${project}/fecal/scrubbed_fecal_table_family.qza
qiime taxa collapse \
    --i-table ${project}/fecal/scrubbed_fecal_table.qza \
    --i-taxonomy ${project}/fecal/taxonomy-fecal.qza \
    --p-level 6 --o-collapsed-table ${project}/fecal/scrubbed_fecal_table_genera.qza
qiime tools export \
    --input-path ${project}/fecal/scrubbed_fecal_table.qza \
    --output-path ${project}/fecal/table_asv
qiime tools export \
    --input-path ${project}/fecal/scrubbed_fecal_table_phyla.qza \
    --output-path ${project}/fecal/table_phyla
qiime tools export \
    --input-path ${project}/fecal/scrubbed_fecal_table_family.qza \
    --output-path ${project}/fecal/table_family
qiime tools export \
    --input-path ${project}/fecal/scrubbed_fecal_table_genera.qza \
    --output-path ${project}/fecal/table_genera
biom convert -i ${project}/fecal/table_asv/feature-table.biom \
    -o ${project}/fecal/feature-table-asv.tsv --to-tsv
biom convert -i ${project}/fecal/table_phyla/feature-table.biom \
    -o ${project}/fecal/feature-table-phyla.tsv --to-tsv
biom convert -i ${project}/fecal/table_family/feature-table.biom \
    -o ${project}/fecal/feature-table-family.tsv --to-tsv
biom convert -i ${project}/fecal/table_genera/feature-table.biom \
    -o ${project}/fecal/feature-table-genera.tsv --to-tsv


qiime feature-table relative-frequency \
    --i-table ${project}/fecal/scrubbed_fecal_table.qza \
    --output-dir ${project}/fecal/ra_table_asv
qiime feature-table relative-frequency \
    --i-table ${project}/fecal/scrubbed_fecal_table_phyla.qza \
    --output-dir ${project}/fecal/ra_table_phyla
qiime feature-table relative-frequency \
    --i-table ${project}/fecal/scrubbed_fecal_table_family.qza \
    --output-dir ${project}/fecal/ra_table_family
qiime feature-table relative-frequency \
    --i-table ${project}/fecal/scrubbed_fecal_table_genera.qza \
    --output-dir ${project}/fecal/ra_table_genera
qiime tools export \
    --input-path ${project}/fecal/ra_table_asv/relative_frequency_table.qza \
    --output-path ${project}/fecal/ra_table_asv
qiime tools export \
    --input-path ${project}/fecal/ra_table_phyla/relative_frequency_table.qza \
    --output-path ${project}/fecal/ra_table_phyla
qiime tools export \
    --input-path ${project}/fecal/ra_table_family/relative_frequency_table.qza \
    --output-path ${project}/fecal/ra_table_family
qiime tools export \
    --input-path ${project}/fecal/ra_table_genera/relative_frequency_table.qza \
    --output-path ${project}/fecal/ra_table_genera
biom convert -i ${project}/fecal/ra_table_asv/feature-table.biom \
    -o ${project}/fecal/feature-table-asv-relab.tsv --to-tsv
biom convert -i ${project}/fecal/ra_table_phyla/feature-table.biom \
    -o ${project}/fecal/feature-table-phyla-relab.tsv --to-tsv
biom convert -i ${project}/fecal/ra_table_family/feature-table.biom \
    -o ${project}/fecal/feature-table-family-relab.tsv --to-tsv
biom convert -i ${project}/fecal/ra_table_genera/feature-table.biom \
    -o ${project}/fecal/feature-table-genera-relab.tsv --to-tsv

####Soil
qiime tools export --input-path ${project}/soil/core-metrics-results-3900/shannon_vector.qza \
    --output-path ${project}/soil/shannon
qiime tools export --input-path ${project}/soil/core-metrics-results-3900/evenness_vector.qza \
    --output-path ${project}/soil/evenness
qiime tools export --input-path ${project}/soil/core-metrics-results-3900/faith_pd_vector.qza \
    --output-path ${project}/soil/faith_pd
qiime tools export --input-path ${project}/soil/core-metrics-results-3900/observed_features_vector.qza \
    --output-path ${project}/soil/observed
cp ${project}/soil/shannon/alpha-diversity.tsv ${project}/soil/shannon-alpha-diversity.tsv
cp ${project}/soil/evenness/alpha-diversity.tsv ${project}/soil/evenness-alpha-diversity.tsv
cp ${project}/soil/faith_pd/alpha-diversity.tsv ${project}/soil/faithpd-alpha-diversity.tsv
cp ${project}/soil/observed/alpha-diversity.tsv ${project}/soil/observed-alpha-diversity.tsv

qiime tools export \
    --input-path ${project}/soil/core-metrics-results-3900/unweighted_unifrac_distance_matrix.qza \
    --output-path ${project}/soil/core-metrics-results-3900/extracted_unweighted_unifrac
cp ${project}/soil/core-metrics-results-3900/extracted_unweighted_unifrac/distance-matrix.tsv \
    ${project}/soil/unweighted-distance-matrix-3900.tsv
qiime tools export \
    --input-path ${project}/soil/core-metrics-results-3900/weighted_unifrac_distance_matrix.qza \
    --output-path ${project}/soil/core-metrics-results-3900/extracted_weighted_unifrac
cp ${project}/soil/core-metrics-results-3900/extracted_weighted_unifrac/distance-matrix.tsv \
    ${project}/soil/weighted-distance-matrix-3900.tsv
qiime tools export \
    --input-path ${project}/soil/core-metrics-results-3900/bray_curtis_distance_matrix.qza \
    --output-path ${project}/soil/core-metrics-results-3900/extracted_bray_curtis
cp ${project}/soil/core-metrics-results-3900/extracted_bray_curtis/distance-matrix.tsv \
    ${project}/soil/bray-distance-matrix-3900.tsv
qiime tools export \
    --input-path ${project}/soil/core-metrics-results-3900/jaccard_distance_matrix.qza \
    --output-path ${project}/soil/core-metrics-results-3900/extracted_jaccard
cp ${project}/soil/core-metrics-results-3900/extracted_jaccard/distance-matrix.tsv \
    ${project}/soil/jaccard-distance-matrix-3900.tsv

qiime taxa collapse \
    --i-table ${project}/soil/filtered_soil_table_il_2023.qza \
    --i-taxonomy ${project}/fecal/taxonomy-fecal.qza \
    --p-level 2 --o-collapsed-table ${project}/soil/filtered_soil_table_il_2023_phyla.qza
qiime taxa collapse \
    --i-table ${project}/soil/filtered_soil_table_il_2023.qza \
    --i-taxonomy ${project}/fecal/taxonomy-fecal.qza \
    --p-level 5 --o-collapsed-table ${project}/soil/filtered_soil_table_il_2023_family.qza
qiime taxa collapse \
    --i-table ${project}/soil/filtered_soil_table_il_2023.qza \
    --i-taxonomy ${project}/fecal/taxonomy-fecal.qza \
    --p-level 6 --o-collapsed-table ${project}/soil/filtered_soil_table_il_2023_genera.qza
qiime tools export \
    --input-path ${project}/soil/filtered_soil_table_il_2023.qza \
    --output-path ${project}/soil/table_asv
qiime tools export \
    --input-path ${project}/soil/filtered_soil_table_il_2023_phyla.qza \
    --output-path ${project}/soil/table_phyla
qiime tools export \
    --input-path ${project}/soil/filtered_soil_table_il_2023_family.qza \
    --output-path ${project}/soil/table_family
qiime tools export \
    --input-path ${project}/soil/filtered_soil_table_il_2023_genera.qza \
    --output-path ${project}/soil/table_genera
biom convert -i ${project}/soil/table_asv/feature-table.biom \
    -o ${project}/soil/feature-table-asv.tsv --to-tsv
biom convert -i ${project}/soil/table_phyla/feature-table.biom \
    -o ${project}/soil/feature-table-phyla.tsv --to-tsv
biom convert -i ${project}/soil/table_family/feature-table.biom \
    -o ${project}/soil/feature-table-family.tsv --to-tsv
biom convert -i ${project}/soil/table_genera/feature-table.biom \
    -o ${project}/soil/feature-table-genera.tsv --to-tsv


qiime feature-table relative-frequency \
    --i-table ${project}/soil/filtered_soil_table_il_2023.qza \
    --output-dir ${project}/soil/ra_table_asv
qiime feature-table relative-frequency \
    --i-table ${project}/soil/filtered_soil_table_il_2023_phyla.qza \
    --output-dir ${project}/soil/ra_table_phyla
qiime feature-table relative-frequency \
    --i-table ${project}/soil/filtered_soil_table_il_2023_family.qza \
    --output-dir ${project}/soil/ra_table_family
qiime feature-table relative-frequency \
    --i-table ${project}/soil/filtered_soil_table_il_2023_genera.qza \
    --output-dir ${project}/soil/ra_table_genera
qiime tools export \
    --input-path ${project}/soil/ra_table_asv/relative_frequency_table.qza \
    --output-path ${project}/soil/ra_table_asv
qiime tools export \
    --input-path ${project}/soil/ra_table_phyla/relative_frequency_table.qza \
    --output-path ${project}/soil/ra_table_phyla
qiime tools export \
    --input-path ${project}/soil/ra_table_family/relative_frequency_table.qza \
    --output-path ${project}/soil/ra_table_family
qiime tools export \
    --input-path ${project}/soil/ra_table_genera/relative_frequency_table.qza \
    --output-path ${project}/soil/ra_table_genera
biom convert -i ${project}/soil/ra_table_asv/feature-table.biom \
    -o ${project}/soil/feature-table-asv-relab.tsv --to-tsv
biom convert -i ${project}/soil/ra_table_phyla/feature-table.biom \
    -o ${project}/soil/feature-table-phyla-relab.tsv --to-tsv
biom convert -i ${project}/soil/ra_table_family/feature-table.biom \
    -o ${project}/soil/feature-table-family-relab.tsv --to-tsv
biom convert -i ${project}/fecal/ra_table_genera/feature-table.biom \
    -o ${project}/soil/feature-table-genera-relab.tsv --to-tsv
